# Introduction to JupyterLab

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
songs = pd.read_csv("data.csv")
songs.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,Unnamed: 17
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4,0.286,1,Mask Off,Future,NaN
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4,0.588,1,Redbone,Childish Gambino,NaN
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4,0.173,1,Xanny Family,Future,NaN
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4,0.230,1,Master Of None,Beach House,NaN
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4,0.904,1,Parallel Lines,Junior Boys,NaN


In [ ]:
vec=songs.iloc[:,1:15]    #making a vector with only  numerical values i.e. skipping all the string values("artist" and "song_title")
vec.shape
vec.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4,0.286,1
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4,0.588,1
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4,0.173,1
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4,0.230,1
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4,0.904,1


In [ ]:
sim=cosine_similarity(vec)  #calculating the cosine similarity corresponding to each song
sim.shape   #gives a matrix with each row denoting similarity between songs as
# sim[1][100] => GIVES THE SIMILARITY SCORE BETWEEN SONG 1 AND SONG 100.

(2017, 2017)

In [ ]:
#finds song ID i.e index of song in the csv
def find_songId(title):
    song_id=songs[songs.song_title==title]["id"].values[0]
    return song_id

#finds reccomendations using a given song title
def find_recommendations(title):
    song_id=find_songId(title)
    scores=list(enumerate(sim[song_id]))
    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)  #sorts all the songs in the list in reverse order (decreasing order)
    sorted_scores=sorted_scores[1:]                               #skips the first index as it is the same song with highest similarity
    rec_songs=[]
    for i in sorted_scores:
        indx=i[0]
        rec_songs.append(songs.loc[indx]["song_title"])       #adds all song title according to the scores found
    return rec_songs                                          # returns the songs

In [ ]:
def recommend_ten(songs_list):          #fetches only the first 10 songs with highest similarity
    first_ten=[]
    count=0
    for song in songs_list:
        if count > 9:
            break
        count+=1
        first_ten.append(song)
    return first_ten

In [ ]:
lst=find_recommendations("Redbone")     #example taken
m=recommend_ten(lst)



In [ ]:
m


['Lollipop',
 'Ohio - filous Remix',
 'The Last Goodbye',
 'History',
 'Whatcha Gonna Do',
 'Pacifier',
 'Borderline',
 'Save Tonight',
 'Solicitude',
 'Check']

In [ ]:
find_songId(m[0])


1187

In [ ]:
songs.loc[1]    #original songs

id                                 1
acousticness                   0.199
danceability                   0.743
duration_ms                   326933
energy                         0.359
instrumentalness             0.00611
key                                1
liveness                       0.137
loudness                     -10.401
mode                               1
speechiness                   0.0794
tempo                        160.083
time_signature                     4
valence                        0.588
target                             1
song_title                   Redbone
artist              Childish Gambino
Unnamed: 17                      NaN
Name: 1, dtype: object

In [ ]:
songs.loc[1187] #song with the highest similarity

id                       1187
acousticness           0.0656
danceability            0.828
duration_ms            299333
energy                  0.433
instrumentalness     0.000876
key                         0
liveness                0.122
loudness               -9.716
mode                        1
speechiness             0.199
tempo                 148.073
time_signature              4
valence                 0.446
target                      0
song_title           Lollipop
artist              Lil Wayne
Unnamed: 17               NaN
Name: 1187, dtype: object